In [1]:
import torch

from seq2seq.data import Dataset
from seq2seq.model import Seq2seqModel
from seq2seq.train import Trainer, Evaluator

In [2]:
GPU_ID = 1
device = torch.device("cuda:{}".format(GPU_ID)) if torch.cuda.is_available() else torch.device("cpu")

# 1. Preprocess Data

In [3]:
SRC_FILE_PATH = 'smart_mobile/smart_src_khaiii.txt'
TGT_FILE_PATH = 'smart_mobile/smart_tgt_khaiii.txt'

EXPR_PATH = 'smart_mobile/'

BATCH_SIZE = 32
MAX_SRC_LEN = 10
MAX_TGT_LEN = 10

In [4]:
%%time
dataset = Dataset(
    src_file_path=SRC_FILE_PATH,
    tgt_file_path=TGT_FILE_PATH,
    max_src_len=MAX_SRC_LEN,
    max_tgt_len=MAX_TGT_LEN
)

Reading lines...
Read 90729 sentence pairs

Trim data to 20074 sentence pairs
Avg length of src :  7.272591411776427
Avg length of tgt :  7.307063863704294

Source vocab : 5062 (0 reduced)
Target vocab : 5060 (0 reduced)

Success to preprocess data!

CPU times: user 895 ms, sys: 128 ms, total: 1.02 s
Wall time: 1.02 s


# 2. Define Model

In [5]:
NUM_LAYERS = 1
INPUT_SIZE = dataset.src_vocab_size
EMBED_SIZE = 64
HIDDEN_SIZE = 64
OUTPUT_SIZE = dataset.tgt_vocab_size

In [6]:
model = Seq2seqModel(
    n_layers=NUM_LAYERS,
    input_size=INPUT_SIZE,
    emb_size=EMBED_SIZE,
    hidden_size=HIDDEN_SIZE,
    output_size=OUTPUT_SIZE,
    max_tgt_len=MAX_TGT_LEN,
    dropout_p=0.0,
    bi_encoder=True,
    device=device
)

In [7]:
model

Seq2seqModel(
  (encoder): EncoderRNN(
    (embedding): Embedding(5062, 64, padding_idx=0)
    (rnn): GRU(64, 64, batch_first=True, bidirectional=True)
  )
  (decoder): DecoderRNN(
    (embedding): Embedding(5060, 64, padding_idx=0)
    (rnn): GRU(64, 128, batch_first=True)
    (attn): Attn(
      (W): Linear(in_features=256, out_features=128, bias=True)
    )
    (out): Linear(in_features=128, out_features=5060, bias=True)
    (softmax): LogSoftmax()
  )
)

# 3. Train Model

In [8]:
#trainer = Trainer(
#    model,
#    dataset,
#    device=device,
#    print_interval=1,
#    plot_interval=-1,
#    checkpoint_interval=10,
#    expr_path=EXPR_PATH
#)

In [9]:
#trainer.train(num_epoch=10, batch_size=BATCH_SIZE)

# 4. Evaluate Model

In [10]:
evaluator = Evaluator(dataset, model, device=device)
evaluator.loadModel(EXPR_PATH+'ep10.model')

In [13]:
pairs, attn_list = evaluator.evalModel(num=100, beam_size=5)
for p, attn in zip(pairs, attn_list):
    print('Input : ' + ' '.join(p[0]))
    print('Gen   : ' + ' '.join(p[1]))
    print()

/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Input : 그렇 어요 ? 괜찮 네요 ? <EOS>
Gen   : 담당자 이 많 으시 ㄴ가요 ? <EOS>

Input : 공중화장실 이용 금액 은 무료 이 ㅂ니다 . <EOS>
Gen   : 그렇 군요 . 감사 하 ㅂ니다 . <EOS>

Input : 재발행 하 려면 얼마나 걸리 나요 ? <EOS>
Gen   : 10 분 정도 걸리 ㅂ니다 . <EOS>

Input : 사이 다가 좋 겠 네요 . <EOS>
Gen   : 알 겠 습니다 . <EOS>

Input : 이거 하나 도 모르 겠 어요 . <EOS>
Gen   : 알 겠 습니다 . <EOS>

Input : 실례 하 ㅂ니다 . 안 에 계시 ㅂ니까 ? <EOS>
Gen   : 무슨 일 이 시 죠 ? <EOS>

Input : 무슨 일 로 빌리 시 려고 하 죠 ? <EOS>
Gen   : 여행 중 이 시 ㄴ가요 ? <EOS>

Input : 실례 하 ㅂ니다 . <EOS>
Gen   : 무슨 일 이 시 죠 ? <EOS>

Input : 뭐 ㄹ 요 . <EOS>
Gen   : @@ 일본 @@ 입 니다 . <EOS>

Input : 뭐 ㄹ 요 . <EOS>
Gen   : @@ 일본 @@ 입 니다 . <EOS>

Input : 감사 하 ㅂ니다 . <EOS>
Gen   : 즐겁 ㄴ 여행 되 시 어요 . <EOS>

Input : 대사관 에 연락 하 여 보 겠 습니다 . <EOS>
Gen   : 네 , 감사 하 ㅂ니다 . <EOS>

Input : 실례 하 ㅂ니다 . <EOS>
Gen   : 무슨 일 이 시 죠 ? <EOS>

Input : 네 , 명 이 요 . <EOS>
Gen   : 알 겠 습니다 . <EOS>

Input : 머리 를 자르 려고 하 는데요 . <EOS>
Gen   : 예 , 말씀 하 시 어요 . <EOS>

Input : 네 , 바로 하 여 드리 겠 습니다 . <EOS>
Gen   : 감사 하 ㅂ니다 . <EOS>

Input : 로 스 팅 정도 는 어떻 ㄴ가요 ? <EOS>
Gen   : 중